<div class="alert alert-block alert-success">

# Case Study 📓: Customer Segmentation and Targeting
    
[Case Study Link](https://docs.google.com/document/d/1AvUydV8er-qNNTwTlH8ITMLpX-29t14wc3wvFQX2goY/edit?usp=sharing)


</div>

##  <font color= darkred> Import modules and data </font>

In [3]:
# data manipulation
import pandas as pd
import numpy as np

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# data viz
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight") # styling the charts

import warnings
warnings.filterwarnings('ignore')

**Load Data**

In [4]:
url = 'https://raw.githubusercontent.com/vkoul/data/main/misc/household_data.csv'

df = pd.read_csv(url)

Hypothesis

* Dummies for categorical 
* Standardize the data 
* Correlation 
* County vs cluster group
* Outliers

<div class="alert alert-block alert-success">

# Reference material 📚


* [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/): good book for hands on practice and codes. 
* [ISLR](https://www.statlearning.com/)- good coverage of the theory

* [Kmeans Categorical](https://towardsdatascience.com/clustering-on-numerical-and-categorical-features-6e0ebcf1cbad)
* [Clustering Mixed Data](https://www.tomasbeuzen.com/post/clustering-mixed-data/)     

</div>